In [1]:
from mlens.ensemble import SuperLearner, BlendEnsemble
import pickle
from sklearn.ensemble import VotingClassifier
import pandas as pd
import numpy as np
from scipy.optimize import minimize, fsolve
from time import time
import datetime

[MLENS] backend: threading


In [2]:
SEED = 21

In [161]:
train = pd.read_feather("transform/train_transform_after_preparation.feather")
pred = pd.read_feather("transform/test_transform_after_preparation.feather")

In [162]:
X_train = train.drop(['SalePrice', 'Id'], 1)
y_train = np.log1p(train['SalePrice'])

/tmp/ipykernel_7887/2050573941.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = train.drop(['SalePrice', 'Id'], 1)


In [163]:
X_pred = pred.drop('Id', 1)

/tmp/ipykernel_7887/2827053377.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_pred = pred.drop('Id', 1)


In [87]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.235294,0.207668,0.033420,0.666667,0.500,0.050725,0.116667,0.12250,0.322669,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000000,0.255591,0.038795,0.555556,0.875,0.246377,0.566667,0.00000,0.446984,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.235294,0.217252,0.046507,0.666667,0.500,0.065217,0.133333,0.10125,0.222121,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.294118,0.191693,0.038561,0.666667,0.500,0.688406,0.666667,0.00000,0.098720,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.235294,0.268371,0.060576,0.777778,0.500,0.072464,0.166667,0.21875,0.299360,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
y_true = train['SalePrice'].values

In [5]:
y_true 

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [6]:
y_true = np.log1p(y_true)

In [7]:
y_true 

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

# Load models

In [8]:
ridge_file = "ridge/ridge_best.pkl"
lasso_file = "lasso/lasso_best.pkl"

In [9]:
with open(ridge_file, 'rb') as f:
    ridge = pickle.load(f)
    oof_ridge_load = pickle.load(f)
    sub_ridge_load = pickle.load(f)

In [10]:
with open(lasso_file, 'rb') as f:
    lasso = pickle.load(f)
    oof_lasso_load = pickle.load(f)
    sub_lasso_load = pickle.load(f)

In [11]:
ridge

Ridge(alpha=3.7491487176400535, max_iter=935, tol=0.001150285538917668)

In [12]:
oof_ridge_load

array([206696.71088582, 203628.90215836, 212714.18238885, ...,
       263412.68263122, 140071.00141104, 144364.25831351])

In [13]:
oof_ridge = np.log1p(oof_ridge_load)

In [14]:
oof_ridge

array([12.23901267, 12.22405942, 12.26770938, ..., 12.48148101,
       11.84991187, 11.88010188])

In [15]:
sub_ridge_load

array([102983.04688615, 126591.4673688 , 151232.69914907, ...,
       138305.25731862, 101363.34819557, 177360.26153891])

In [16]:
lasso

Lasso(alpha=0.00033138399899106596, max_iter=1009, selection='random',
      tol=0.0008715067546363353)

In [17]:
oof_lasso_load

array([207329.40328459, 199685.15818334, 214600.73838566, ...,
       265231.76294223, 136129.37827068, 146244.70410202])

In [18]:
oof_lasso = np.log1p(oof_lasso_load)

In [19]:
oof_lasso

array([12.24206895, 12.2045022 , 12.27653921, ..., 12.48836307,
       11.82136837, 11.89304339])

In [20]:
sub_lasso_load

array([105554.20345355, 133963.26207084, 154487.27174914, ...,
       141217.61786711, 106014.25892907, 184012.21909543])

In [21]:
oof = np.zeros((2, y_true.shape[0]))

In [24]:
oof[0] = oof_ridge
oof[1] = oof_lasso
oof

array([[12.23901267, 12.22405942, 12.26770938, ..., 12.48148101,
        11.84991187, 11.88010188],
       [12.24206895, 12.2045022 , 12.27653921, ..., 12.48836307,
        11.82136837, 11.89304339]])

In [25]:
def get_models():
    models = []
    models.append(ridge)
    models.append(lasso)
    return models

In [26]:
models = get_models()

# Voting Classifier

In [27]:
#clf = [rf, et, knn, svc, rg]
#eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')

# Weighted Averaging

In [28]:
from sklearn.metrics import mean_squared_error

In [64]:
def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = 1)
    #oof_blend = weights * oof
    return mean_squared_error(y_true, oof_blend)

In [65]:
"""def grad_func(weights):
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof[i], np.zeros((oof.shape[1]))
        c += weights[i] * oof[i]
        gradients[i] = 1.0 / oof.shape[0] * (a-c) * (-a)
    return gradients"""

'def grad_func(weights):\n    gradients = np.zeros(oof.shape[0])\n    for i in range(oof.shape[0]):\n        a, b, c = y_true, oof[i], np.zeros((oof.shape[1]))\n        c += weights[i] * oof[i]\n        gradients[i] = 1.0 / oof.shape[0] * (a-c) * (-a)\n    return gradients'

In [66]:
neg_mean_square_scores = {}
for i, key in enumerate(oof):
    score_oof = mean_squared_error(y_true, key)
    neg_mean_square_scores[i] = score_oof
    print(f'{i+1} CV MSE:\t', score_oof)
    print(f'{i+1} CV RMSE:\t', np.sqrt(score_oof))

1 CV MSE:	 0.01362452459012408
1 CV RMSE:	 0.11672413884935746
2 CV MSE:	 0.012320876866073579
2 CV RMSE:	 0.11099944534128799


In [67]:
init_guess = [1 / oof.shape[0]] * oof.shape[0]
init_guess

[0.5, 0.5]

In [68]:
bnds = [(0, 1) for _ in range(oof.shape[0])]
bnds

[(0, 1), (0, 1)]

In [71]:
oof

array([[12.23901267, 12.22405942, 12.26770938, ..., 12.48148101,
        11.84991187, 11.88010188],
       [12.24206895, 12.2045022 , 12.27653921, ..., 12.48836307,
        11.82136837, 11.89304339]])

In [72]:
oof_blend2 = np.tensordot(init_guess, oof, axes = 1)
oof_blend2

array([12.24054081, 12.21428081, 12.27212429, ..., 12.48492204,
       11.83564012, 11.88657264])

In [77]:
# tol = 1e-10
init_guess = [1 / oof.shape[0]] * oof.shape[0]
bnds = [(0, 1) for _ in range(oof.shape[0])]
'''cons = {'type': 'eq', 
        'fun': lambda x: np.sum(x) - 1, 
        'jac': lambda x: [1] * len(x)}
'''
print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()
res_scipy = minimize(fun = func_numpy_metric, 
                     x0 = init_guess, 
                     method = 'nelder-mead', 
                     #jac = grad_func, # grad_func 
                     bounds = bnds, 
                     #constraints = cons, 
                     #tol = tol,
                     options={'xatol': 1e-8, 'disp': True})
print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
print('RMSE Optimised Blend OOF:', np.sqrt(res_scipy.fun))
print('Optimised Weights:', res_scipy.x)

Inital Blend OOF: 0.012788533627330364
Optimization terminated successfully.
         Current function value: 0.012321
         Iterations: 18
         Function evaluations: 34
[00:00] Optimised Blend OOF: 0.012320876866073579
RMSE Optimised Blend OOF: 0.11099944534128799
Optimised Weights: [0. 1.]


In [74]:
print('Check the sum of all weights:', np.sum(res_scipy.x))
if np.sum(res_scipy.x) - 1 <= tol:
    print('Great! The sum of all weights equals to 1!')
else:
    print('Manual adjustion is needed to modify the weights.')

Check the sum of all weights: 1.0
Great! The sum of all weights equals to 1!


# Stacking

In [309]:
from sklearn.linear_model import LinearRegression
from mlens.metrics import make_scorer
from sklearn.metrics import mean_squared_error

In [295]:
def scoring(y_true, y_pred):
    return -mean_squared_error(y_true, y_pred)

In [324]:
stacking = SuperLearner(scorer=scoring, folds=10, shuffle=True, sample_size=len(X_train), random_state=SEED, verbose=1)

In [325]:
stacking.add(models)

SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...3b4700>)],
   n_jobs=-1, name='group-48', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       verbose=1)

In [326]:
stacking.add_meta(LinearRegression())

SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...3b4700>)],
   n_jobs=-1, name='group-49', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       verbose=1)

In [327]:
X_train.to_numpy()

array([[0.23529412, 0.20766773, 0.0334198 , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.25559105, 0.03879502, ..., 0.        , 1.        ,
        0.        ],
       [0.23529412, 0.2172524 , 0.04650728, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.29411765, 0.21086262, 0.03618687, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.2172524 , 0.03934189, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [328]:
y_train.to_numpy()

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [329]:
stacking.fit(X_train.to_numpy(), y_train.to_numpy())


Fitting 2 layers
Fit complete                        | 00:00:00


SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...3b4700>)],
   n_jobs=-1, name='group-49', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       verbose=1)

In [330]:
mse = mean_squared_error(y_train.to_numpy(), stacking.predict(X_train.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


In [331]:
mse

0.009635365364576407

In [332]:
rmse = np.sqrt(mse)
rmse

0.09815989692627232

In [318]:
stacking_pred = np.expm1(stacking.predict(X_pred.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


# Blending

In [335]:
blending = BlendEnsemble(test_size=0.5, shuffle=True, scorer=scoring, sample_size=len(X_train), random_state=SEED, verbose=1)

In [336]:
blending.add(models)

BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...3b4700>)],
   n_jobs=-1, name='group-50', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       test_size=0.5, verbose=1)

In [337]:
blending.add_meta(LinearRegression())

BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...3b4700>)],
   n_jobs=-1, name='group-51', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       test_size=0.5, verbose=1)

In [338]:
blending.fit(X_train.to_numpy(), y_train.to_numpy())


Fitting 2 layers
Fit complete                        | 00:00:00


BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...3b4700>)],
   n_jobs=-1, name='group-51', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f16693b4700>, shuffle=True,
       test_size=0.5, verbose=1)

In [339]:
mse = mean_squared_error(y_train.to_numpy(), blending.predict(X_train.to_numpy()))
mse


Predicting 2 layers
Predict complete                    | 00:00:00


0.01010472361349041

In [340]:
rmse = np.sqrt(mse)
rmse

0.10052225431958045

In [341]:
blending_pred = np.expm1(stacking.predict(X_pred.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


In [342]:
blending_pred

array([105836.5 , 135387.5 , 155100.  , ..., 141615.9 , 106927.33,
       185440.38], dtype=float32)

# Final Submission

In [320]:
# LB 0.19652
ens = stacking_pred
submission = pd.DataFrame()
submission['Id'] = pred['Id']
submission["SalePrice"] = ens
submission.to_csv("submit_ensemble.csv", index=False)